# 0 - Construction de la base de données

## Importation des modules

In [1]:
# Modules de base
import os
import json
import pandas as pd
import sys
import yaml

# Ajout du chemin
sys.path.append('..')

# Importation des modules ad hoc
from dashboard_template_database.builders.schema import SchemaBuilder
from dashboard_template_database.builders.tables import DuckdbTablesBuilder
from dashboard_template_database.loaders.local.loader import Loader

# Chargement du fichier de configurations
with open("../config.yaml") as file:
    config = yaml.safe_load(file)

# Chargement du fichier de parmaètres
with open("../parameters/labels.json") as file:
    labels = json.load(file)


## Importation des données

In [2]:
# Initialisation du loader
loader = Loader()
# Importation des données
df_origin = loader.load(os.path.join('../', config['INPUT_DATA']))
# Conversion en datetime
df_origin['date'] = pd.to_datetime(df_origin['date'])
df_origin.head()

,indicator,country,date,value,kind,horizon,week,model,training
0,Gross Domestic Product,France,1960-04-01,0.375710,observed,NaN,NaN,NaN,NaN
1,Gross Domestic Product,France,1960-07-01,0.748561,observed,NaN,NaN,NaN,NaN
2,Gross Domestic Product,France,1960-10-01,1.185218,observed,NaN,NaN,NaN,NaN
3,Gross Domestic Product,France,1961-01-01,1.608374,observed,NaN,NaN,NaN,NaN
4,Gross Domestic Product,France,1961-04-01,1.600329,observed,NaN,NaN,NaN,NaN


## Construction du schéma

### Initialisation de la classe

In [3]:
# Initialisation du schéma
schema_builder = SchemaBuilder(df=df_origin, categorical_threshold=config['THRESHOLD'])

### Construction des méta-données

In [4]:
# Construction du jeu de métadonnées
df_metadata = schema_builder.create_metadata_table(column_labels=labels)

df_metadata.head()

2025-01-08 22:32:45,797 - INFO - Successfully extracted meta-data from column 'indicator'
2025-01-08 22:32:45,839 - INFO - The column 'indicator' is of type 'object' and the number of modalities 2 satisfies the categorical threshold criteria 200
2025-01-08 22:32:45,840 - INFO - Successfully extracted meta-data from column 'country'
2025-01-08 22:32:45,864 - INFO - The column 'country' is of type 'object' and the number of modalities 6 satisfies the categorical threshold criteria 200
2025-01-08 22:32:45,866 - INFO - Successfully extracted meta-data from column 'date'
2025-01-08 22:32:45,866 - INFO - Successfully extracted meta-data from column 'value'
2025-01-08 22:32:45,867 - INFO - Successfully extracted meta-data from column 'kind'
2025-01-08 22:32:45,892 - INFO - The column 'kind' is of type 'object' and the number of modalities 4 satisfies the categorical threshold criteria 200
2025-01-08 22:32:45,893 - INFO - Successfully extracted meta-data from column 'horizon'
2025-01-08 22:32:

,name,label,python_type,sql_type,is_categorical
0,country,Country,object,VARCHAR,True
1,date,Date,datetime64[ns],TIMESTAMP,False
2,horizon,Horizon,float64,DOUBLE,False
3,indicator,Indicator,object,VARCHAR,True
4,kind,Kind,object,VARCHAR,True


### Construction des tables de dimensions

In [5]:
# Construction des types de dimensions
dimension_tables = schema_builder.create_dimension_tables(column_labels=labels)
dimension_tables['indicator'].head()

2025-01-08 22:32:45,946 - INFO - Successfully extracted meta-data from column 'indicator'
2025-01-08 22:32:45,986 - INFO - The column 'indicator' is of type 'object' and the number of modalities 2 satisfies the categorical threshold criteria 200
2025-01-08 22:32:45,989 - INFO - Successfully extracted meta-data from column 'country'
2025-01-08 22:32:46,015 - INFO - The column 'country' is of type 'object' and the number of modalities 6 satisfies the categorical threshold criteria 200
2025-01-08 22:32:46,017 - INFO - Successfully extracted meta-data from column 'date'
2025-01-08 22:32:46,018 - INFO - Successfully extracted meta-data from column 'value'
2025-01-08 22:32:46,018 - INFO - Successfully extracted meta-data from column 'kind'
2025-01-08 22:32:46,044 - INFO - The column 'kind' is of type 'object' and the number of modalities 4 satisfies the categorical threshold criteria 200
2025-01-08 22:32:46,045 - INFO - Successfully extracted meta-data from column 'horizon'
2025-01-08 22:32:

,value,label
0,0,Gross Domestic Product
1,1,Private Consumption


### Construction de la table d'information

In [6]:
# Construction de la table d'informations
df_fact = schema_builder.create_fact_table(column_labels=labels)
df_fact.head()

C:\Users\bolli\OneDrive\Documents\COD Code\dashboard-template-database\dashboard_template_database\builders\schema.py:197: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df_fact[column] = self.df_fact[column].replace(dict_label_value)
2025-01-08 22:32:46,514 - INFO - Successfully replace modalities by ids in column 'country'
C:\Users\bolli\OneDrive\Documents\COD Code\dashboard-template-database\dashboard_template_database\builders\schema.py:197: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df_fact[column] = self.df_fact[

,indicator,country,date,value,kind,horizon,week,model,training
0,0,0,1960-04-01,0.375710,0,NaN,NaN,0.0,0.0
1,0,0,1960-07-01,0.748561,0,NaN,NaN,0.0,0.0
2,0,0,1960-10-01,1.185218,0,NaN,NaN,0.0,0.0
3,0,0,1961-01-01,1.608374,0,NaN,NaN,0.0,0.0
4,0,0,1961-04-01,1.600329,0,NaN,NaN,0.0,0.0


### Création de l'ensemble des tables

In [7]:
# Création de l'ensemble des tables du schéma
df_metadata, dimension_tables, df_fact = schema_builder.build(column_labels=labels)

2025-01-08 22:32:47,220 - INFO - Successfully extracted meta-data from column 'indicator'
2025-01-08 22:32:47,264 - INFO - The column 'indicator' is of type 'object' and the number of modalities 2 satisfies the categorical threshold criteria 200
2025-01-08 22:32:47,265 - INFO - Successfully extracted meta-data from column 'country'
2025-01-08 22:32:47,294 - INFO - The column 'country' is of type 'object' and the number of modalities 6 satisfies the categorical threshold criteria 200
2025-01-08 22:32:47,295 - INFO - Successfully extracted meta-data from column 'date'
2025-01-08 22:32:47,296 - INFO - Successfully extracted meta-data from column 'value'
2025-01-08 22:32:47,298 - INFO - Successfully extracted meta-data from column 'kind'
2025-01-08 22:32:47,325 - INFO - The column 'kind' is of type 'object' and the number of modalities 4 satisfies the categorical threshold criteria 200
2025-01-08 22:32:47,326 - INFO - Successfully extracted meta-data from column 'horizon'
2025-01-08 22:32:

## Construction de la base de données

### Initialisation du builder

In [8]:
# Initialisation du builder
builder = DuckdbTablesBuilder(df=df_origin, categorical_threshold=config['THRESHOLD'], path=os.path.join('../', config['OUTPUT_DATA']))

### Création du schéma

In [9]:
# Construction du schéma duckDB
builder.build_duckdb_schema()

2025-01-08 22:32:49,073 - INFO - Successfully extracted meta-data from column 'indicator'
2025-01-08 22:32:49,126 - INFO - The column 'indicator' is of type 'object' and the number of modalities 2 satisfies the categorical threshold criteria 200
2025-01-08 22:32:49,128 - INFO - Successfully extracted meta-data from column 'country'
2025-01-08 22:32:49,165 - INFO - The column 'country' is of type 'object' and the number of modalities 6 satisfies the categorical threshold criteria 200
2025-01-08 22:32:49,166 - INFO - Successfully extracted meta-data from column 'date'
2025-01-08 22:32:49,168 - INFO - Successfully extracted meta-data from column 'value'
2025-01-08 22:32:49,169 - INFO - Successfully extracted meta-data from column 'kind'
2025-01-08 22:32:49,207 - INFO - The column 'kind' is of type 'object' and the number of modalities 4 satisfies the categorical threshold criteria 200
2025-01-08 22:32:49,209 - INFO - Successfully extracted meta-data from column 'horizon'
2025-01-08 22:32:

### Affichage du schéma

In [10]:
# Affichage du schéma
builder.display_schema()

2025-01-08 22:32:52,405 - INFO - 
 Created Tables:
2025-01-08 22:32:52,406 - INFO - 
 dim_country Structure:
2025-01-08 22:32:52,408 - INFO -   value: BIGINT
2025-01-08 22:32:52,409 - INFO -   label: VARCHAR
2025-01-08 22:32:52,410 - INFO - 
 dim_indicator Structure:
2025-01-08 22:32:52,411 - INFO -   value: BIGINT
2025-01-08 22:32:52,412 - INFO -   label: VARCHAR
2025-01-08 22:32:52,413 - INFO - 
 dim_kind Structure:
2025-01-08 22:32:52,414 - INFO -   value: BIGINT
2025-01-08 22:32:52,419 - INFO -   label: VARCHAR
2025-01-08 22:32:52,420 - INFO - 
 dim_model Structure:
2025-01-08 22:32:52,422 - INFO -   value: BIGINT
2025-01-08 22:32:52,423 - INFO -   label: VARCHAR
2025-01-08 22:32:52,424 - INFO - 
 dim_training Structure:
2025-01-08 22:32:52,427 - INFO -   value: BIGINT
2025-01-08 22:32:52,428 - INFO -   label: VARCHAR
2025-01-08 22:32:52,429 - INFO - 
 fact_table Structure:
2025-01-08 22:32:52,431 - INFO -   indicator: BIGINT
2025-01-08 22:32:52,434 - INFO -   country: BIGINT
2025-

### Exemple de requête

In [11]:
# Requête de la table d'information
print(builder.conn.execute("SELECT * FROM dim_model").fetchall())

[(1, 'ElasticNetCV'), (2, 'ExtraTrees'), (3, 'LassoCV'), (4, 'RandomForest'), (5, 'RidgeCV'), (6, 'XGBStandard'), (0, None)]
